In [1]:
import pennylane as qml
from pennylane import numpy as np
from numpy.linalg import qr

In [2]:
def qr_haar(N):
    """Generate a Haar-random matrix using the QR decomposition."""
    # Step 1
    A, B = np.random.normal(size=(N, N)), np.random.normal(size=(N, N))
    Z = A + 1j * B

    # Step 2
    Q, R = qr(Z)

    # Step 3
    Lambda = np.diag([R[i, i] / np.abs(R[i, i]) for i in range(N)])

    # Step 4
    return np.dot(Q, Lambda)

def haar_integral(num_qubits, samples):

    """
        Return calculation of Haar Integral for a specified number of samples.

    """
    N = 2**num_qubits
    randunit_density = np.zeros((N, N), dtype=complex)

    zero_state = np.zeros(N, dtype=complex)

    zero_state[0] = 1

    for _ in range(samples):
      A = np.matmul(zero_state, qr_haar(N)).reshape(-1,1)
      randunit_density += np.kron(A, A.conj().T) 

    randunit_density/=samples
    return randunit_density

In [3]:
import pennylane as qml
import numpy as np

def pqc_integral(num_qubits, ansatze, size, samples,num_layers):
    """
    Return calculation of Integral for a PQC over the uniformly sampled 
    parameters θ for the specified number of samples.
    """
    N = num_qubits
    randunit_density = np.zeros((2**N, 2**N), dtype=complex)

    for _ in range(samples):
        params = np.random.uniform(-np.pi, np.pi, size)

        # Use PennyLane QNode for circuit definition and execution
        @qml.qnode(dev)
        def quantum_circuit(params):
            ansatze(params, N,num_layers)
            return qml.state()

        # Execute the PennyLane quantum circuit
        state_vector = quantum_circuit(params)

        # Reshape the state vector
        U = state_vector.reshape(-1, 1)

        # Update the density matrix
        randunit_density += np.kron(U, U.conj().T)

    return randunit_density / samples

# Define your ansatz function using PennyLane operations
def uqc_circuit(params, num_qubits, num_layers):
    for l in range(num_layers - 1):
        [qml.RX(params[l, 0, i], wires=i) for i in range(num_qubits)]
        [qml.RY(params[l, 1, i], wires=i) for i in range(num_qubits)]
        [qml.CZ(wires=[i, i + 1]) for i in range(num_qubits - 1)]
        qml.CZ(wires=[num_qubits - 1, 0])
    [qml.RX(params[num_layers - 1, 0, i], wires=i) for i in range(num_qubits)]
    [qml.RY(params[num_layers - 1, 1, i], wires=i) for i in range(num_qubits)]

def idle_circuit(params, num_qubits, num_layers):
    for l in range(num_layers):
        [qml.Identity(wires=i) for i in range(num_qubits)]

def more_expressive_circuit(params, num_qubits, num_layers):
    for l in range(num_layers):
        [qml.Hadamard(wires=i) for i in range(num_qubits)]
        [qml.RZ(params[l, 0, i], wires=i) for i in range(num_qubits)]


In [4]:
num_qubits = 4
num_layers = 5
size = (num_layers,2,num_qubits)
samples = 2048
dev = qml.device("default.qubit", wires=num_qubits)

np.linalg.norm(haar_integral(4, 2048) - pqc_integral(num_qubits, uqc_circuit, size, samples,num_layers))

0.029033873474706845

In [5]:
num_qubits = 1
num_layers = 1
size = (num_layers,1,num_qubits)
samples = 2048
dev = qml.device("default.qubit", wires=num_qubits)

np.linalg.norm(haar_integral(1, 2048) - pqc_integral(num_qubits, idle_circuit, size, samples,num_layers))

0.6984199850008671

In [6]:
num_qubits = 1
num_layers = 1
size = (num_layers,1,num_qubits)
samples = 2048
dev = qml.device("default.qubit", wires=num_qubits)

np.linalg.norm(haar_integral(1, 2048) - pqc_integral(num_qubits, more_expressive_circuit, size, samples,num_layers))

0.021175089002383025

In [7]:
np.linalg.norm(haar_integral(2, 2048) - haar_integral(2, 2048))

0.021041845157198445

In [2]:
import numpy as np
from scipy.linalg import qr

def qr_haar(N):
    """Generate a Haar-random matrix using the QR decomposition."""
    # Step 1
    A, B = np.random.normal(size=(N, N)), np.random.normal(size=(N, N))
    Z = A + 1j * B

    # Step 2
    Q, R = qr(Z)

    # Step 3
    Lambda = np.diag([R[i, i] / np.abs(R[i, i]) for i in range(N)])

    # Step 4
    return np.dot(Q, Lambda)

def haar_integral(num_qubits, samples):

    N = 2**num_qubits
    randunit_density = np.zeros((N**2, N**2), dtype=complex)

    zero_state = np.zeros(N, dtype=complex)
    zero_state[0] = 1

    for _ in range(samples):
        A = np.matmul(zero_state, qr_haar(N)).reshape(-1, 1)
        B = np.matmul(zero_state, qr_haar(N)).reshape(-1, 1)
        
        # Update the calculation to compute the outer product and then take the Kronecker product
        randunit_density += np.kron(np.outer(A, A.conj()), np.outer(B, B.conj()))

    randunit_density /= samples
    return randunit_density

np.linalg.norm(haar_integral(1, 2048) - haar_integral(1, 2048))

0.034228608438357254

In [4]:
import numpy as np
import pennylane as qml

# Assuming you have defined your device 'dev' somewhere in your code

def pqc_integral_2(num_qubits, ansatze, size, samples, num_layers):
    """
    Return calculation of Integral for a PQC over the uniformly sampled 
    parameters θ for the specified number of samples.
    """
    N = 2**num_qubits
    randunit_density = np.zeros((N**2, N**2), dtype=complex)

    for _ in range(samples):
        params_1 = np.random.uniform(-np.pi, np.pi, size)
        #params_2 = np.random.uniform(-np.pi, np.pi, size)

        # Use PennyLane QNode for circuit definition and execution
        @qml.qnode(dev)
        def quantum_circuit(params):
            ansatze(params, num_qubits, num_layers)
            return qml.state()

        # Execute the PennyLane quantum circuit for the first set of parameters
        state_vector_1 = quantum_circuit(params_1)

        # Execute the PennyLane quantum circuit for the second set of parameters
        state_vector_2 = quantum_circuit(params_1)

        # Reshape the state vectors
        U_1 = state_vector_1.reshape(-1, 1)
        U_2 = state_vector_2.reshape(-1, 1)

        # Update the density matrix
        #randunit_density += np.kron(U_1, U_1.conj().T) @ np.kron(U_2, U_2.conj().T)
        randunit_density += np.kron(np.outer(U_1, U_1.conj()), np.outer(U_2, U_2.conj()))

    return randunit_density / samples

# Define your ansatz function using PennyLane operations
def uqc_circuit(params, num_qubits, num_layers):
    for l in range(num_layers - 1):
        [qml.RX(params[l, 0, i], wires=i) for i in range(num_qubits)]
        [qml.RY(params[l, 1, i], wires=i) for i in range(num_qubits)]
        [qml.CZ(wires=[i, i + 1]) for i in range(num_qubits - 1)]
        qml.CZ(wires=[num_qubits - 1, 0])
    [qml.RX(params[num_layers - 1, 0, i], wires=i) for i in range(num_qubits)]
    [qml.RY(params[num_layers - 1, 1, i], wires=i) for i in range(num_qubits)]

def idle_circuit(params, num_qubits, num_layers):
    for l in range(num_layers):
        [qml.Identity(wires=i) for i in range(num_qubits)]

# Example usage:
num_qubits = 4
num_layers = 5
size = (num_layers, 2, num_qubits)
samples = 2048
dev = qml.device("default.qubit", wires=num_qubits)

np.linalg.norm(haar_integral(num_qubits, 2048) - pqc_integral_2(num_qubits, uqc_circuit, size, samples, num_layers))


0.06706083493313106

In [21]:
np.linalg.norm(haar_integral(1, 2048) - haar_integral(1,2048))

0.029708737345564666

In [12]:
np.linalg.norm(haar_integral(4, 2048) - pqc_integral_2(num_qubits, idle_circuit, size, samples, num_layers))**2

0.9962977582523471